In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
# read files
health_camp_details = pd.read_csv("./Train_2/Train/Health_Camp_Detail.csv")
patient_profile = pd.read_csv("./Train_2/Train/Patient_Profile.csv")
first_health_camp = pd.read_csv("./Train_2/Train/First_Health_Camp_Attended.csv")
second_health_camp = pd.read_csv("./Train_2/Train/Second_Health_Camp_Attended.csv")
third_health_camp = pd.read_csv("./Train_2/Train/Third_Health_Camp_Attended.csv")

train = pd.read_csv("./Train_2/Train/Train.csv")

In [62]:
test = pd.read_csv("./test_l0Auv8Q.csv")

# EDA

In [63]:
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [52]:
test.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [36]:
health_camp_details.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [41]:
patient_profile.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [54]:
print(patient_profile["Patient_ID"].nunique())
print(train["Patient_ID"].nunique())
print(test["Patient_ID"].nunique())

37633
29828
15324


In [162]:
first_health_camp.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [39]:
second_health_camp.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [40]:
third_health_camp.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [64]:
# create a new dataframe to play around with the data without affecting source
working_data = train.copy()

In [82]:
def merge(working_data):
    """To merge health camp details and profile info"""
    a=pd.merge(working_data,health_camp_details,on="Health_Camp_ID",how="left") #merge health camp details
    b=pd.merge(a, patient_profile,on="Patient_ID",how="left") #merge profile info of patient
    return b

In [165]:
def merge_healthscore(cols):
    """Puts 1 if health score exist 0 if not"""
    ID = cols[0]
    health_ID = cols[1]
    c_type = cols[2]
    score = 0
    
    if c_type =='First':
        if ID in first_health_camp["Patient_ID"].values:
            if health_ID in first_health_camp[first_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                    score=1
        
    elif c_type =='Second':
        if ID in second_health_camp["Patient_ID"].values:
            if health_ID in second_health_camp[second_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                    score=1
                    
    elif c_type =='Third':
        score="NA"
    
    return score

In [166]:
def merge_stall(cols):
    """Return 1 if stall visited 0 if not"""
    ID = cols[0]
    health_ID = cols[1]
    c_type = cols[2]
    score = 0
    if c_type == "Third":
        if ID in third_health_camp["Patient_ID"].values:
            if health_ID in third_health_camp[third_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                score=1
    else:
        score="NA"
    return score

In [114]:
working_data=merge(working_data)

In [167]:
working_data["Health_Score_exists"]=working_data[["Patient_ID","Health_Camp_ID","Category1"]].apply(merge_healthscore,axis=1)

In [168]:
working_data["Stall_visited"]=working_data[["Patient_ID","Health_Camp_ID","Category1"]].apply(merge_stall,axis=1)

In [170]:
# create a column favourable 1 if yes 0 if no
def create_favourable(cols):
    score_exists=cols[0]
    stall_visited=cols[1]
    
    ret=0
    if score_exists !="NA":
        if score_exists==1:
            ret=1
        elif score_exists==0:
            ret=0
    elif score_exists=="NA":
        if stall_visited==1:
            ret=1
        elif stall_visited ==0:
            ret = 0
    return ret

In [171]:
working_data["Favourable"]=working_data[["Health_Score_exists","Stall_visited"]].apply(create_favourable,axis=1)

In [175]:
working_data.drop(["Health_Score_exists","Stall_visited"],axis=1,inplace=True)

In [177]:
working_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Patient_ID         75278 non-null  int64 
 1   Health_Camp_ID     75278 non-null  int64 
 2   Registration_Date  74944 non-null  object
 3   Var1               75278 non-null  int64 
 4   Var2               75278 non-null  int64 
 5   Var3               75278 non-null  int64 
 6   Var4               75278 non-null  int64 
 7   Var5               75278 non-null  int64 
 8   Camp_Start_Date    75278 non-null  object
 9   Camp_End_Date      75278 non-null  object
 10  Category1          75278 non-null  object
 11  Category2          75278 non-null  object
 12  Category3          75278 non-null  int64 
 13  Online_Follower    75278 non-null  int64 
 14  LinkedIn_Shared    75278 non-null  int64 
 15  Twitter_Shared     75278 non-null  int64 
 16  Facebook_Shared    75278 non-null  int64